# Import Library

In [1]:
from collections import OrderedDict
import pickle

from ml_pee_stego_v3 import MLPEEStego

# Set up the secret

In [2]:
from utils.data_preparation import get_filenames_from_folder, get_secret_file

# Tentukan folder path tempat secret
SECRET_FOLDER_PATH = 'keys/bin'

# Dapatkan semua nama file .txt dari folder
secret_filenames = get_filenames_from_folder('txt', SECRET_FOLDER_PATH)
secret_filenames.sort(reverse=True)
print(secret_filenames)

# Masukkan ke dalam dictionary berisi nama filename beserta isi secret filenya
secret_dicts = OrderedDict((secret_filename, get_secret_file(
    SECRET_FOLDER_PATH + '/' + secret_filename + '.txt')) for secret_filename in secret_filenames)
# print(secret_dicts.get(secret_filenames[1]))

for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")

['secret_0.99_bps', 'secret_0.66_bps', 'secret_0.51_bps', 'secret_0.48_bps', 'secret_0.39_bps', 'secret_0.32_bps', 'secret_0.21_bps', 'secret_0.15_bps', 'secret_0.08_bps']
Secret untuk secret_0.99_bps
Secret untuk secret_0.66_bps
Secret untuk secret_0.51_bps
Secret untuk secret_0.48_bps
Secret untuk secret_0.39_bps
Secret untuk secret_0.32_bps
Secret untuk secret_0.21_bps
Secret untuk secret_0.15_bps
Secret untuk secret_0.08_bps


# Set up the model

In [3]:
import warnings
from sklearn.exceptions import InconsistentVersionWarning

# Menonaktifkan peringatan InconsistentVersionWarning
warnings.simplefilter("ignore", category=InconsistentVersionWarning)

models = OrderedDict()

models['SVR'] = 'models/svr_model.pkl'
models['SVR 2.0'] = 'models/svr_model_2.0.pkl'
models['LASSO'] = 'models/lasso_model.pkl'
models['ElasticNet'] = 'models/elastic_net_model.pkl'
models['Bayesian Ridge'] = 'models/bayesian_ridge_model.pkl'
models['SGD Regressor'] = 'models/sgd_model.pkl'

# Pick sample patient

In [4]:
from utils.data_preparation import get_original_data

patient_code = '100'
original_data = get_original_data(patient_code=patient_code)

# Run model trial

In [5]:
# Example
model_name = 'LASSO'
with open(models.get(model_name), 'rb') as model_file:
    model = pickle.load(model_file)

stego = MLPEEStego(model)
payload_rate = 2
threshold = 0


for secret_name, secret_data in secret_dicts.items():
    print(f"Secret untuk {secret_name}")
    print(
        f"Model {model_name} secret {secret_name} payload_rate {payload_rate} threshold {threshold}")
    print(">>Embedding...")
    watermarked_data, mirror_data, last_phase, last_i, last_embedded_bit_total, result = stego.embed(
        original_data, secret_data, payload_rate=payload_rate, threshold=threshold)
    print(
        f"Last Phase: {last_phase}, Last I: {last_i}, Last embedded bit: {last_embedded_bit_total}")
    print(">>Extracting...")
    extracted_original_data, extracted_secret_data = stego.extract(
        watermarked_data, mirror_data, last_phase, last_i, last_embedded_bit_total, payload_rate=payload_rate, threshold=threshold)

    print(">>", original_data[0:20], secret_data[0:20])
    print(">>", extracted_original_data[0:20],
          extracted_secret_data[0:20])
    print(">>", "Len -> Ori:", len(original_data), ", Ex-ori:", len(extracted_original_data),
            ", Sec:", len(
                secret_data), ", Ex-sec:", len(extracted_secret_data))
    print("******\n******")

Secret untuk secret_0.99_bps
Model LASSO secret secret_0.99_bps payload_rate 2 threshold 0
>>Embedding...
True
[-145 -145 -145 -145 -145 -145 -145 -145 -119 -137]
Total yang kurang dari sama dengan threshold (T = 1): 423
Unhidden secret: 0
NCC: 0.9999806303469385
PRD: 0.2923959408421488
SNR: 50.680573214454014
PSNR: 29.580281678318137
Time: 0.7492210865020752
Last Phase: 3, Last I: 143, Last embedded bit: 2
>>Extracting...
[-145 -145 -145 -145 -145 -145 -145 -145 -119 -137]
[-145 -145 -145 -145 -145 -145 -145 -145 -119 -135]
[-145 -145 -145 -145 -145 -145 -145 -145 -120 -135]
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11101111011001110011
>> [-145 -145 -145 -145 -145 -145 -145 -145 -120 -135 -145 -150 -160 -155
 -160 -175 -180 -185 -170 -155] 11101111011001110011
>> Len -> Ori: 3600 , Ex-ori: 3600 , Sec: 3564 , Ex-sec: 3564
******
******
Secret untuk secret_0.66_bps
Model LASSO secret secret_0.66_bps payload_rate 2 threshol

In [6]:
secret_data = "110101100010010011001111110011000110001111011000011111110111111011011101010010110100000101100011000101001100100001111010101000000101100011101110000110111010110110110000101011000010001110000010011100001011100000100110010010011001010100101100011000100110110000100100010111010100110100010000"
secret_data[287:288]

'0'